In [1]:
import numpy as np



## Day1

https://adventofcode.com/2024/day/1

In [2]:
#Part 1
data = np.loadtxt("./puzzle_files/aoc1.txt")
print(np.abs(np.sort(data[:,0]) - np.sort(data[:,1])).sum())

2815556.0


In [3]:
# Part 2
ans = 0
for k in data[:,0]:
    ans += k*(data[:,1] == k).sum()
print(ans)

23927637.0


## Day 2

https://adventofcode.com/2024/day/2

In [4]:
with open("./puzzle_files/aoc2.txt", "r") as f:
    lines = f.readlines()
    #print(lines)

def report_safe(dd):
    return (((dd == np.sort(dd)).all() or (dd == np.sort(dd)[::-1]).all()) and (np.abs(np.diff(dd)) <= 3).all() and (np.abs(np.diff(dd)) >= 1).all())

In [5]:
# Part 1
ans = 0
for line in lines:
    dd = np.array(line.split(), dtype=np.float64)
    if report_safe(dd):
        ans += 1

print(ans)

230


In [6]:
# Part 2
ans = 0
for line in lines:
    dd = np.array(line.split(), dtype=np.float64)
    if report_safe(dd):
        ans += 1
    else:
        for i in range(len(dd)):
            dd2 = np.delete(np.copy(dd), i)
            if report_safe(dd2):
                ans += 1
                break

print(ans)

301


## Day 3

In [7]:
with open("./puzzle_files/aoc3.txt", "r") as f:
    lines = f.readlines()

In [43]:
import re
# Part 1
ans = 0
for line in lines:
    muls = re.findall("mul\(\d+,\d+\)", line)
    nums = np.array([mul[4:-1].split(',') for mul in muls], dtype=np.uint64)
    ans += (nums.prod(axis=1)).sum()
print(ans)

170778545.0


In [82]:
# Part 2
ans = 0
doflag = True
for line in lines:
    muls = re.findall("(mul\(\d+,\d+\))|(do\(\))|(don\'t\(\))", line)
    mul_array = []
    for mul in muls:
        if len(mul[1])+len(mul[2]) > 0:
            assert len(mul[0]) == 0
            if mul[1] == 'do()':
                doflag=True
            elif mul[2] == "don't()":
                doflag=False
            else:
                raise RunTimeError
        else:
            assert len(mul[0]) > 0
            if doflag:
                mul_array.append(mul[0][4:-1].split(','))

    ans += np.array(mul_array, dtype=np.uint64).prod(axis=1).sum()
print(ans)


82868252.0
